In [9]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


Install Handwrite recognition requirements

In [10]:
!pip install -r "/content/drive/MyDrive/handwritten-text-recognition/requirements.txt"

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [11]:
!pip install -U Werkzeug==0.16.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [12]:
!apt install poppler-utils

Reading package lists... Done
Building dependency tree       
Reading state information... Done
poppler-utils is already the newest version (0.62.0-2ubuntu2.12).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 42 not upgraded.


In [13]:
%cd /content/drive/MyDrive/handwritten-text-recognition

/content/drive/MyDrive/handwritten-text-recognition


In [14]:
from pdf2image import convert_from_path
import cv2
import glob
import numpy as np
from PIL import Image
import difflib
import importlib
import math
import random
import string

import gluonnlp as nlp
import leven
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import mxnet as mx
from skimage import transform as skimage_tf, exposure
from tqdm import tqdm

from ocr.utils.expand_bounding_box import expand_bounding_box
from ocr.utils.sclite_helper import ScliteHelper
from ocr.utils.word_to_line import sort_bbs_line_by_line, crop_line_images
from ocr.utils.iam_dataset import IAMDataset, resize_image, crop_image, crop_handwriting_page
from ocr.utils.encoder_decoder import Denoiser, ALPHABET, encode_char, decode_char, EOS, BOS
from ocr.utils.beam_search import ctcBeamSearch

import ocr.utils.denoiser_utils
import ocr.utils.beam_search

importlib.reload(ocr.utils.denoiser_utils)
from ocr.utils.denoiser_utils import SequenceGenerator

importlib.reload(ocr.utils.beam_search)
from ocr.utils.beam_search import ctcBeamSearch


from ocr.paragraph_segmentation_dcnn import SegmentationNetwork, paragraph_segmentation_transform
from ocr.word_and_line_segmentation import SSD as WordSegmentationNet, predict_bounding_boxes
from ocr.handwriting_line_recognition import Network as HandwritingRecognitionNet, handwriting_recognition_transform
from ocr.handwriting_line_recognition import decode as decoder_handwriting, alphabet_encoding
from matplotlib.patches import Rectangle

In [15]:
#greedy algorithm convert the output of the handwriting recognition network into strings
def get_arg_max(prob): 
    arg_max = prob.topk(axis=2).asnumpy()
    return decoder_handwriting(arg_max)[0]

def get_beam_search(prob, width=5):
    possibilities = ctcBeamSearch(prob.softmax()[0].asnumpy(), alphabet_encoding, None, width)
    return possibilities[0]

def get_denoised(prob, ctc_bs=False):
    if ctc_bs: # Using ctc beam search before denoising yields only limited improvements a is very slow
        text = get_beam_search(prob)
    else:
        text = get_arg_max(prob)
    src_seq, src_valid_length = encode_char(text)
    src_seq = mx.nd.array([src_seq], ctx=ctx)
    src_valid_length = mx.nd.array(src_valid_length, ctx=ctx)
    encoder_outputs, _ = denoiser.encode(src_seq, valid_length=src_valid_length)
    states = denoiser.decoder.init_state_from_encoder(encoder_outputs, 
                                                      encoder_valid_length=src_valid_length)
    inputs = mx.nd.full(shape=(1,), ctx=src_seq.context, dtype=np.float32, val=BOS)
    output = generator.generate_sequences(inputs, states, text)[::2]
    return output.strip()
ctx = mx.gpu(0) if mx.context.num_gpus() > 0 else mx.cpu()

In [16]:
#One time execute
paragraph_segmentation_net = SegmentationNetwork(ctx=ctx)
paragraph_segmentation_net.cnn.load_parameters("models/paragraph_segmentation2.params", ctx=ctx)

paragraph_segmentation_net.hybridize()

word_segmentation_net = WordSegmentationNet(2, ctx=ctx)
word_segmentation_net.load_parameters("models/word_segmentation2.params")
word_segmentation_net.hybridize()

handwriting_line_recognition_net = HandwritingRecognitionNet(rnn_hidden_states=512,
                                                             rnn_layers=2, ctx=ctx, max_seq_len=160)
handwriting_line_recognition_net.load_parameters("models/handwriting_line8.params", ctx=ctx)
handwriting_line_recognition_net.hybridize()

FEATURE_LEN = 150
denoiser = Denoiser(alphabet_size=len(ALPHABET), max_src_length=FEATURE_LEN, max_tgt_length=FEATURE_LEN, num_heads=16, embed_size=256, num_layers=2)
denoiser.load_parameters('models/denoiser2.params', ctx=ctx)

denoiser.hybridize(static_alloc=True)

language_model, vocab = nlp.model.big_rnn_lm_2048_512(dataset_name='gbw', vocab='/content/drive/MyDrive/gbw-ebb1a287.vocab', pretrained=True, ctx=ctx)
moses_tokenizer = nlp.data.SacreMosesTokenizer()
moses_detokenizer = nlp.data.SacreMosesDetokenizer()

beam_sampler = nlp.model.BeamSearchSampler(beam_size=20,
                                           decoder=denoiser.decode_logprob,
                                           eos_id=EOS,
                                           scorer=nlp.model.BeamSearchScorer(),
                                           max_length=150)

generator = SequenceGenerator(beam_sampler, language_model, vocab, ctx, moses_tokenizer, moses_detokenizer)



/usr/local/lib/python3.7/dist-packages/gluonnlp/model/utils.py:268: UserWarning: Both dataset_name and vocab are specified. Loading vocab based on dataset_name. Input "vocab" argument will be ignored.
  warnings.warn('Both dataset_name and vocab are specified. '


Vocab file is not found. Downloading.


In [17]:
def handrecog(fname):
  #Saving pages in jpeg format
  cnt=0
  answer=[]
  f="/content/drive/MyDrive/answer sheets pdf/"+fname
  pages = convert_from_path(f, 500)
  for i,page in enumerate(pages):
    y="/content/drive/MyDrive/images/"+str(i)+"im.jpeg"
    page.save(y, 'JPEG')
    cnt+=1
  print("No.of pages:",cnt)
  images = []
  files = glob.glob ("/content/drive/MyDrive/images/*.jpeg")
  for myFile in files:
      print(myFile)
      image = Image.open(myFile)
      new_image = image.resize((2479, 3542))
      new_image.save(myFile)
      image = cv2.imread(myFile, cv2.IMREAD_GRAYSCALE)
      images.append (image)
  imag=np.array(images) 
  print(' shape:', imag.shape)
  random.seed(123)
  random.seed(1)
  plt.figure(figsize=(10,10)) # specifying the overall grid size

  for i in range(cnt):
      plt.subplot(2,2,i+1)    # the number of images in the grid is 5*5 (25)
      plt.imshow(imag[i], cmap='Greys_r')

  form_size = (1120, 800)

  predicted_bbs = []
  plt.figure(figsize=(10,10)) # specifying the overall grid size
  #fig, axs = plt.subplots(int(len(imag)/2), 2, figsize=(15, 9 * len(imag)/2))
  for i, image in enumerate(imag):
    #s_y, s_x = int(i/2), int(i%2)
    resized_image = paragraph_segmentation_transform(image, form_size)
    bb_predicted = paragraph_segmentation_net(resized_image.as_in_context(ctx))
    bb_predicted = bb_predicted[0].asnumpy()
    bb_predicted = expand_bounding_box(bb_predicted, expand_bb_scale_x=0.06, expand_bb_scale_y=0.35)
    predicted_bbs.append(bb_predicted)
    
    #axs[s_y, s_x].imshow(image, cmap='Greys_r')
    #axs[s_y, s_x].set_title("{}".format(i))

    (x, y, w, h) = bb_predicted
    image_h, image_w = image.shape[-2:]
    (x, y, w, h) = (x * image_w, y * image_h, w * image_w, h * image_h)
    rect = patches.Rectangle((x, y), w, h, fill=False, color="r", ls="--")
    #axs[s_y, s_x].add_patch(rect)
  

    plt.subplot(2,2,i+1)    # the number of images in the grid is 5*5 (25)
    plt.imshow(imag[i], cmap='Greys_r')
    plt.gca().add_patch(rect)

  segmented_paragraph_size = (700, 700)
  #fig, axs = plt.subplots(int(len(images)/2), 2, figsize=(15, 9 * len(images)/2))
  plt.figure(figsize=(10,10))
  paragraph_segmented_images = []

  for i, image in enumerate(images):
      #s_y, s_x = int(i/2), int(i%2)

      bb = predicted_bbs[i]
      image = crop_handwriting_page(image, bb, image_size=segmented_paragraph_size)
      paragraph_segmented_images.append(image)
      plt.subplot(2,2,i+1)    # the number of images in the grid is 5*5 (25)
      plt.imshow(image, cmap='Greys_r')
      #axs[s_y, s_x].imshow(image, cmap='Greys_r')
      #axs[s_y, s_x].axis('off')

  min_c = 0.1
  overlap_thres = 0.1
  topk = 600
  plt.figure(figsize=(10,10))
  #fig, axs = plt.subplots(int(len(paragraph_segmented_images)/2), 2, figsize=(15, 5 * int(len(paragraph_segmented_images)/2)))
  predicted_words_bbs_array = []

  for i, paragraph_segmented_image in enumerate(paragraph_segmented_images):
      #s_y, s_x = int(i/2), int(i%2)
      predicted_bb = predict_bounding_boxes(word_segmentation_net, paragraph_segmented_image, min_c, overlap_thres, topk, ctx)
      predicted_words_bbs_array.append(predicted_bb)
      #axs[s_y, s_x].imshow(paragraph_segmented_image, cmap='Greys_r')
      plt.subplot(2,2,i+1)    # the number of images in the grid is 5*5 (25)
      plt.imshow(paragraph_segmented_image, cmap='Greys_r')
      for j in range(predicted_bb.shape[0]):     
          (x, y, w, h) = predicted_bb[j]
          image_h, image_w = paragraph_segmented_image.shape[-2:]
          (x, y, w, h) = (x * image_w, y * image_h, w * image_w, h * image_h)
          rect = patches.Rectangle((x, y), w, h, fill=False, color="r")
          plt.gca().add_patch(rect)
          #axs[s_y, s_x].add_patch(rect)
          #axs[s_y, s_x].axis('off')

  line_images_array = []
  #fig, axs = plt.subplots(int(len(paragraph_segmented_images)/2), 2, figsize=(15, 9 * int(len(paragraph_segmented_images)/2)))
  plt.figure(figsize=(10,10))
  for i, paragraph_segmented_image in enumerate(paragraph_segmented_images):
      #s_y, s_x = int(i/2), int(i%2)
      #axs[s_y, s_x].imshow(paragraph_segmented_image, cmap='Greys_r')
      #axs[s_y, s_x].axis('off')
      #axs[s_y, s_x].set_title("{}".format(i))
      plt.subplot(2,2,i+1)    # the number of images in the grid is 5*5 (25)
      plt.imshow(paragraph_segmented_image, cmap='Greys_r')
      predicted_bbs = predicted_words_bbs_array[i]
      line_bbs = sort_bbs_line_by_line(predicted_bbs, y_overlap=0.4)
      line_images = crop_line_images(paragraph_segmented_image, line_bbs)
      line_images_array.append(line_images)
    
      for line_bb in line_bbs:
          (x, y, w, h) = line_bb
          image_h, image_w = paragraph_segmented_image.shape[-2:]
          (x, y, w, h) = (x * image_w, y * image_h, w * image_w, h * image_h)

          rect = patches.Rectangle((x, y), w, h, fill=False, color="r")
          plt.gca().add_patch(rect)


  line_image_size = (60, 800)
  character_probs = []
  for line_images in line_images_array:
      form_character_prob = []
      for i, line_image in enumerate(line_images):
          line_image = handwriting_recognition_transform(line_image, line_image_size)
          line_character_prob = handwriting_line_recognition_net(line_image.as_in_context(ctx))
          form_character_prob.append(line_character_prob)
      character_probs.append(form_character_prob)
  
  for i, form_character_probs in enumerate(character_probs):
    ans=""
    for j, line_character_probs in enumerate(form_character_probs):
        decoded_line_am = get_arg_max(line_character_probs)
        decoded_line_bs = get_beam_search(line_character_probs)
        decoded_line_denoiser = get_denoised(line_character_probs, ctc_bs=False)
        #print("[D ]",decoded_line_denoiser)
        ans+=" "
        ans+=decoded_line_denoiser
    answer.append(ans)
    print("Answer",i+1," :")
    print(answer[i]) 

  return answer

Database and Tables

In [18]:
import sqlite3  
  
con = sqlite3.connect("employee.db")  
print("Database opened successfully")  
  
con.execute('''create table Student(rno INTEGER PRIMARY KEY, studname TEXT NOT NULL, 
                sem TEXT NOT NULL, studpwd TEXT NOT NULL)''')  
con.execute('''CREATE TABLE exam_db ([examid] INTEGER PRIMARY KEY, [subid] text, [totalmark] integer, 
                [sem] text, [noofqs] integer)''')
con.execute('''CREATE TABLE question_db([examid] INTEGER, [qno] integer, [question] text, 
                [modelans] text, [maxmark] integer, PRIMARY KEY(examid,qno))''')
con.execute('''CREATE TABLE score_db ([examid] INTEGER, [rollno] integer, [qno] integer, [answer] text,
                [mark] integer,[evaluated] text  PRIMARY KEY(examid,rollno,qno))''')
con.execute('''CREATE TABLE answer_db([examid] INTEGER,[sem] text, [rollno] integer, [answer] text, 
                [studmark] integer,  PRIMARY KEY(rollno,examid))''')

con.execute('''create table Teacher (teachid INTEGER PRIMARY KEY AUTOINCREMENT, teachname TEXT NOT NULL, 
                subject TEXT NOT NULL, subid TEXT NOT NULL, teachpwd TEXT NOT NULL)''')
  
con.commit()

Database opened successfully


OperationalError: ignored

Show all tables

In [19]:
import sqlite3  
  
con = sqlite3.connect("employee.db") 
r_set=con.execute('''SELECT * from Student''');
print("Student")
for row in r_set:
    print(row)
r_set=con.execute('''SELECT * from Teacher''');
print("Teacher")
for row in r_set:
    print(row)
r_set=con.execute('''SELECT * from exam_db''');
print("exam_db")
for row in r_set:
    print(row)
r_set=con.execute('''SELECT * from question_db''');
print("question_db")
for row in r_set:
    print(row)
r_set=con.execute('''SELECT * from answer_db''');
print("answer_db")
for row in r_set:
    print(row)
r_set=con.execute('''SELECT * from score_db''');
print("score_db")
for row in r_set:
    print(row)
con.commit()

Student
(1, 'meenu', '8', 'meenu')
(2, '8', '8', 'arathy24')
(3, 'Anu', '8', 'anu28')
(4, '8', '8', 'athira29')
(5, 'Jilsha MK', '8', 'jilsha36')
(6, 'Sriannaporani K', '8', 'anna7')
(7, 'Aparna', '8', 'aparna65')
(8, 'Drishya M', '8', 'drishya66')
(9, 'Sonu P', '8', 'sonu69')
(10, '8', '8', 'maya73')
(11, '8', '8', 'charu75')
(12, '8', '8', 'achu')
(13, 'Ashwathi', '8', 'ashwathi77')
Teacher
(1, 'Deepthi VR', 'Machine Learning', 'cs433', 'password')
(2, 'Revathi', 'Social', 'SOC201', 'revathi')
(3, 'Manju', 'OOP', 'cs204', 'manju')
(4, 'Jini ', 'Data Structure', 'cs304', 'jini304')
(5, 'Jeena', 'Geography', 'ce404', 'jeena')
exam_db
(20401, 'cs204', 20, '8', 3)
(20402, 'cs204', 10, '8', 1)
(30401, 'cs304', 20, '8', 2)
(40401, 'ce404', 25, '8', 3)
(43301, 'cs433', 30, '8', 2)
(43302, 'cs433', 40, '8', 4)
(200001, 'SOC201', 40, '1', 2)
question_db
(43301, 1, 'What is machine learning?', 'Machine learning (ML) is the scientific study of algorithms and statistical models that computer sys

In [20]:
cur = con.cursor()  
cur.execute(" update score_db set evaluated='false'")
con.commit()  

In [7]:
import os
from flask import Flask, flash, request, redirect, url_for, render_template
from werkzeug.utils import secure_filename
from flask_ngrok import run_with_ngrok
UPLOAD_FOLDER = '/content/drive/MyDrive/answer sheets pdf'
ALLOWED_EXTENSIONS = {'pdf'}

app = Flask(__name__, template_folder='/content/drive/MyDrive/Colab Notebooks/templates', static_folder='/content/drive/MyDrive/Colab Notebooks/static')
app.config['UPLOAD_FOLDER'] = UPLOAD_FOLDER
run_with_ngrok(app)

In [22]:
@app.route('/', methods=['GET', 'POST'])

def home():
    if request.method == 'POST':
        # check if the post request has the file part
      if request.form['login'] == 'Login':
        return redirect(url_for('login'))
    return '''
    <!doctype html>
    <title>Online Exam Site</title>
    <h1>Welcome... </h1>
    <form method=post enctype=multipart/form-data>
      <input type=submit name=login value=Login>
    </form>
    '''

In [23]:


@app.route('/login', methods=['GET', 'POST'])

def login():
    if request.method == 'POST':
      login.stud=False
      login.name=request.form['name'] 
      pwd=request.form['pwd']
      #check studname & pwd
      #print(login.rno,pwd)
      con = sqlite3.connect("employee.db")
      cur = con.cursor() 
      
     #credential check with DB
      cnt=0
      exam_set=cur.execute('''SELECT * from Student where studname = ? and studpwd = ? ''',(login.name, pwd))
      for row in exam_set:
        login.sem=row[2]
        login.rno=row[0]
        print(row)
        cnt+=1
 
      if cnt==1:
        print("Student Logged in")
        login.stud=True
        return redirect(url_for('student'))
      else:
        #cur1 = con.cursor() 
        teacher_set=cur.execute('''SELECT * from Teacher where teachname = ? and teachpwd = ? ''',(login.name, pwd));
        rows = cur.fetchall()
        cnt=0
        for row in rows:
          print(row)
          cnt+=1
        
        if cnt==1:
          print("Teacher Logged in")
          return redirect(url_for('teacher'))
        else:
          print("No such User...Recheck")
          return redirect(url_for('home'))

      print("Finish")
      con.commit() 
    return '''
    <!DOCTYPE html>
<html>

    <head>
    <title>homepage</title>
    <style>
body {
  background-image: url('https://www.creativefabrica.com/wp-content/uploads/2020/03/08/Back-to-school-background-as-blank-copy-Graphics-3411986-1.jpg');
  background-repeat: no-repeat;
  background-attachment: fixed;  
  background-size: cover;
  font-family: 'Courier New', monospace;
}
.button {
  background-color: #4CAF50;
  border: none;
  color: white;
  padding: 15px 32px;
  text-align: center;
  text-decoration: none;
  display: inline-block;
  font-size: 16px;
  margin: 4px 2px;
  cursor: pointer;
}

input[type=text] {
  width: 30%;
  padding: 12px 20px;
  margin: 8px 0;
  box-sizing: border-box;
}
input[type=password] {
  width: 30%;
  padding: 12px 20px;
  margin: 8px 0;
  box-sizing: border-box;
}
</style>
    </head>
    <body>

    <br><br><br><br>
    <br><br><br><br><br><br>
      <center>
      <h1 style="color:white">automatic answer evaluation</h1>
      <div>
    <br><br>
      <form method=post enctype=multipart/form-data>
     <input type=text name=name placeholder=Username><br><br>
      <input type=password name=pwd placeholder=Password>
      <br><br>
      <input type=submit class="button" value=Login>
      </form>
      </center>
      </div>
    </body>




</html>
    
    '''

Add Teacher 

In [24]:
@app.route('/register', methods=['GET', 'POST'])
#register for teacher only
def register():
  if request.method == 'POST':
    try:  
      register.teachname=request.form['teachname']
      subject=request.form['subject']
      subid=request.form['subid']
      teachpwd=request.form['teachpwd']
      print(register.teachname,subid,subject,teachpwd)
      #add to database
      with sqlite3.connect("employee.db") as con:  
                cur = con.cursor()  
                cur.execute("INSERT into Teacher (teachname,subid,subject,teachpwd) values (?,?,?,?)",(register.teachname,subid,subject,teachpwd))  
                con.commit()  
                msg = "Teacher successfully Added"  
                print(msg)  
                      
    except:  
      con.rollback()  
      msg = "We can not add the teacher to the list"  
    finally:
      if request.form['submit']=="Register":
        return redirect(url_for('teacher'))
      con.close()
  return '''
    <!doctype html>
    <title>Register</title>
    <center>
<br><br><br><br><br><br><br><br><br><br>
    <h2 style="color:white">Registration </h1>
<style>
body {
  background-image: url('https://www.creativefabrica.com/wp-content/uploads/2020/03/08/Back-to-school-background-as-blank-copy-Graphics-3411986-1.jpg');
  background-repeat: no-repeat;
  background-attachment: fixed;  
  background-size: cover;
  font-family: 'Courier New', monospace;
}
.button {
  background-color: #4CAF50;
  border: none;
  color: white;
  padding: 15px 32px;
  text-align: center;
  text-decoration: none;
  display: inline-block;
  font-size: 16px;
  margin: 4px 2px;
  cursor: pointer;
}

input[type=text] {
  width: 20%;
  padding: 12px 20px;
  margin: 8px 0;
  box-sizing: border-box;
}
input[type=password] {
  width: 30%;
  padding: 12px 20px;
  margin: 8px 0;
  box-sizing: border-box;
}
</style>
<body>
    <form method=post enctype=multipart/form-data>
  <br><br>    <input type=text name=teachname placeholder=Teachername>
      <input type=text name=subject placeholder=Subject>
      <input type=text name=subid placeholder=SubjectCode>
      <input type=text name=teachpwd placeholder=Password>
      <input type=submit name=submit class="button" value=Register>
    </form>
    </body>
    '''

Home page of a Student

In [25]:
@app.route('/student', methods=['GET', 'POST'])
def student():
    if request.method == 'POST':
      if request.form['choice'] == 'UploadAnswer':
        student.examid = request.form['examid']
        con = sqlite3.connect("employee.db")
        con.row_factory = sqlite3.Row  
        cur = con.cursor() 
        id=int(student.examid)
        cur.execute('''SELECT * from question_db where examid = ?''',(id, )); 
        student.rows = cur.fetchall()
        con.commit()
        return redirect(url_for('upload'))
      elif request.form['choice'] == 'Marklist':
        return redirect(url_for('marklist'))
      elif request.form['choice'] == 'Markdistribution':
        return redirect(url_for('Markdistribution'))
      elif request.form['choice'] == 'Scorecard':
        return redirect(url_for('scorecard'))
    return '''
    <!doctype html>
    <title>Student</title>
    <center>
    <br><br><br><br>
    <br><br><br><br><br><br>
    <h1 style="color:white">Student's home page </h1>
    <style>
    .button {
  background-color: #4CAF50;
  border: none;
  color: white;
  padding: 15px 32px;
  text-align: center;
  text-decoration: none;
  display: inline-block;
  font-size: 16px;
  margin: 4px 2px;
  cursor: pointer;
}

input[type=text] {
  width: 30%;
  padding: 12px 20px;
  margin: 8px 0;
  box-sizing: border-box;
}
body {
  background-image: url('https://www.creativefabrica.com/wp-content/uploads/2020/03/08/Back-to-school-background-as-blank-copy-Graphics-3411986-1.jpg');
  background-repeat: no-repeat;
  background-attachment: fixed;  
  background-size: cover;
  font-family: 'Courier New', monospace;
}
</style>
<body>
    <form method=post enctype=multipart/form-data>
    <br>
      <p style="color:white">Enter the Exam ID : </p> 
      <br>
      <input type=text name=examid placeholder=ExamID>
      <input type=submit name=choice class="button" value=UploadAnswer>
      <br><br><br><br><br>
      
      <input type=submit name=choice class="button" value=Markdistribution>
      <input type=submit name=choice class="button" value=Scorecard>
    </form>
</head>
    '''

Scorecard 

In [26]:
#scorecard1

@app.route('/scorecard', methods=['GET', 'POST'])

def scorecard():
    if request.method == 'POST':
      if login.stud == True:
        scorecard.name=login.name #name from login page
      else:
        scorecard.name=teacher.studname
      scorecard.sem=request.form['sem'] 
      
      if request.form['submit']=="Scorecard" :
        return redirect(url_for('scorecardview'))
    return '''
      <!doctype html>
      <title>Scorecard</title>
      <center>
      <br><br><br><br>
    <br><br><br><br><br><br>
      <h2 style="color:white">Scorecard</h1>
      <style>
      .button {
  background-color: #4CAF50;
  border: none;
  color: white;
  padding: 15px 32px;
  text-align: center;
  text-decoration: none;
  display: inline-block;
  font-size: 16px;
  margin: 4px 2px;
  cursor: pointer;
}

input[type=text] {
  width: 30%;
  padding: 12px 20px;
  margin: 8px 0;
  box-sizing: border-box;
}
body {
  background-image: url('https://www.creativefabrica.com/wp-content/uploads/2020/03/08/Back-to-school-background-as-blank-copy-Graphics-3411986-1.jpg');
  background-repeat: no-repeat;
  background-attachment: fixed;  
  background-size: cover;
  font-family: 'Courier New', monospace;
}
</style>
<body>
      <form method=post enctype=multipart/form-data>
      <br><br>
        <input type=text name=sem placeholder=Semester>
        <input type=submit name=submit class="button" value=Scorecard>
      </form>
      </body>
      '''


In [27]:
#scorecard2

@app.route('/scorecardview', methods=['GET', 'POST'])
  
def scorecardview():  
  r=0
  con = sqlite3.connect("employee.db") 
  con.row_factory = sqlite3.Row  
  cur = con.cursor() 
  rollno_set=cur.execute("SELECT * from Student")
  ro = cur.fetchall()
  for row in ro:
    if row[1]==scorecard.name:
      r=row[0]  # getrollnumber
      print("Details",row[1],row[0])
      break
  
  score_set=cur.execute("SELECT * from answer_db where sem=? and rollno=?", (scorecard.sem,r)) 
  rows = cur.fetchall() 
  return render_template("scorecardview.html",rows=rows, studname=scorecard.name, rollno=r, sem=scorecard.sem)
con.commit()

    

Marklist

In [28]:
@app.route('/marklist', methods=['GET', 'POST'])

def marklist():
    l=[]
    if request.method == 'POST':
      marklist.id=request.form['id']
       
      if request.form['submit']=="Marklist" :
        return redirect(url_for('marklistview'))
    return '''
    <!doctype html><br><br><br><br>
    <br><br><br><br><br><br>
    <title>Marklist</title>
    <center>
    <h2 style="color:white">Enter ExamID</h1>
         <style>
      .button {
  background-color: #4CAF50;
  border: none;
  color: white;
  padding: 15px 32px;
  text-align: center;
  text-decoration: none;
  display: inline-block;
  font-size: 16px;
  margin: 4px 2px;
  cursor: pointer;
}

input[type=integer] {
  width: 30%;
  padding: 12px 20px;
  margin: 8px 0;
  box-sizing: border-box;
}
body {
  background-image: url('https://www.creativefabrica.com/wp-content/uploads/2020/03/08/Back-to-school-background-as-blank-copy-Graphics-3411986-1.jpg');
  background-repeat: no-repeat;
  background-attachment: fixed;  
  background-size: cover;
  font-family: 'Courier New', monospace;
}
</style>
<body>
    <form method=post enctype=multipart/form-data><br><br>
      <input type=integer name=id placeholder=examid>
      <input type=submit name=submit class="button" value=Marklist>
    </form>
    </body>
    '''

In [29]:
#marklist 2
@app.route('/marklistview', methods=['GET', 'POST'])

def marklistview():
      con = sqlite3.connect("employee.db")
      con.row_factory = sqlite3.Row  
      cur = con.cursor()
      id=int(marklist.id)
      teachname='not assigned' 
      sub='not assigned'
      subid=0
      sem='not assigned'
      sem_set=cur.execute('''SELECT * from exam_db where examid = ?''', (id, ))
      for row in sem_set:
        subid=row[1]
        print("Semester   : ",row[3])
        sem=row[3]
      sub_set=cur.execute('''SELECT * from Teacher where subid = ?''', (subid, ))
      for row in sub_set:
        print("Subject    : ",row[2])
        sub=row[2]
        print("Professor  : ",row[1])
        teachname=row[1]
     #credential check with DB
      print("Exam ID    : ",id)
      print("RollNo   Mark")
      rollno_set=cur.execute('''SELECT * from answer_db where examid = ?''', (id, ))
      rows = cur.fetchall()
      for row in rows:
        print(row[2],"    ",row[4])
      #return render_template("markdistributionview.html", sum=sum,examid=id,rollno=r,l=l,studname = Markdistribution.name)
      return render_template("marklistview.html", rows=rows, examid=id, teachname=teachname, sub=sub, sem=sem)
con.commit()

Mark Distribution

In [30]:
#markdistribution 1
@app.route('/Markdistribution', methods=['GET', 'POST'])

def Markdistribution():
    if request.method == 'POST':
      if login.stud == True:            #student logged in
        Markdistribution.name=login.name
      else:
        Markdistribution.name=teacher.studname  #teacher logged in
      Markdistribution.id=request.form['id'] 
      
      if request.form['submit']=="Markdistribution" :
        return redirect(url_for('markdistributionview')) 
    return '''
    <!doctype html>
    <title>Markdistribution</title>
    <center>
    <br><br><br><br><br><br><br><br><br><br>
    <h2 style="color:white">Markdistribution</h1>
         <style>
      .button {
  background-color: #4CAF50;
  border: none;
  color: white;
  padding: 15px 32px;
  text-align: center;
  text-decoration: none;
  display: inline-block;
  font-size: 16px;
  margin: 4px 2px;
  cursor: pointer;
}

input[type=integer] {
  width: 30%;
  padding: 12px 20px;
  margin: 8px 0;
  box-sizing: border-box;
}
body {
  background-image: url('https://www.creativefabrica.com/wp-content/uploads/2020/03/08/Back-to-school-background-as-blank-copy-Graphics-3411986-1.jpg');
  background-repeat: no-repeat;
  background-attachment: fixed;  
  background-size: cover;
  font-family: 'Courier New', monospace;
}
</style>
<body>
    <form method=post enctype=multipart/form-data><br><br>
      <input type=integer name=id placeholder=ExamID>
      <input type=submit name=submit class="button" value=Markdistribution>
    </form>
    </body>
    '''

In [31]:
#markdistribution 2
@app.route('/markdistributionview', methods=['GET', 'POST'])

def markdistributionview():
      con = sqlite3.connect("employee.db")
      con.row_factory = sqlite3.Row  
      cur = con.cursor() 
      id=int(Markdistribution.id)
      r_set=cur.execute('''SELECT * from Student where studname = ?''',(Markdistribution.name, ));

      for row in r_set:
        r=row[0]  #rollnumber
        print(r)
        print(id)
     #credential check with DB
      sum=0
      i=1
      l=[]
      r_set=cur.execute('''SELECT * from score_db where rollno = ? and examid = ?''',(r, id));
      rows = cur.fetchall()
      for row in rows:
        l.append(row[4])
        print(row)
        print("MARK FOR ANSWER",i," - ",row[4])
        sum+=row[4]
        i+=1
      print("Total - ",sum)
      #return render_template("markdistributionview.html", sum=sum,examid=id,rollno=r,l=l,studname = Markdistribution.name)
      return render_template("markdistributionview.html",who=login.stud, rows = rows, sum = sum, studname = Markdistribution.name, examid = id )
con.commit()  

Home page of a teacher

In [32]:
@app.route('/teacher', methods=['GET', 'POST'])
def teacher():
  #stud add
  #exam add
  #marklist view=teacher's students marks
  #scorecard
    if request.method == 'POST':
      if request.form['choice'] == 'AddStudent':
        return redirect(url_for('addstud'))
      elif request.form['choice'] == 'AddTeacher':
        return redirect(url_for('register'))
      elif request.form['choice'] == 'AddExam':
        return redirect(url_for('addexam'))
      elif request.form['choice'] == 'Marklist':
        teacher.subid=request.form['subid']
        return redirect(url_for('marklist'))

      elif request.form['choice'] == 'Markdistribution':
        teacher.studname=request.form['studname']
        return redirect(url_for('Markdistribution'))
      elif request.form['choice'] == 'Scorecard':
        teacher.studname=request.form['studname']
        return redirect(url_for('scorecard'))
      elif request.form['choice'] == 'Evaluate':
        return redirect(url_for('score'))
    return '''
    <!doctype html>

    <title>Teacher</title>
    <br><br><br><br><br><br><br><br><br><br>
    <center>
    <h1 style="color:white">Teacher's home page</h1>
         <style>
      .button {
  background-color: #4CAF50;
  border: none;
  color: white;
  padding: 15px 32px;
  text-align: center;
  text-decoration: none;
  display: inline-block;
  font-size: 16px;
  margin: 4px 2px;
  cursor: pointer;
}

input[type=text] {
  width: 20%;
  padding: 12px 20px;
  margin: 8px 0;
  box-sizing: border-box;
}
body {
  background-image: url('https://www.creativefabrica.com/wp-content/uploads/2020/03/08/Back-to-school-background-as-blank-copy-Graphics-3411986-1.jpg');
  background-repeat: no-repeat;
  background-attachment: fixed;  
  background-size: cover;
  font-family: 'Courier New', monospace;
}
</style>
<body>
    <form method=post enctype=multipart/form-data>
    <br><br>
      <input type=submit name=choice class="button" value=AddStudent>

      <input type=submit name=choice class="button" value=AddTeacher>
      <input type=submit name=choice class="button" value=AddExam>
      <br><br><br>
<p style="color:white">Enter SubjectID for MARKLIST</p>
<br><br>

       <input type=text name=subid placeholder=SubjectID>
      <input type=submit name=choice class="button" value=Marklist>
<br><br><br>
<p style="color:white">Enter the name of Student for Scorecard/Markdistribution </p>
<br><br>
       <input type=text name=studname placeholder=StudentName>
      <input type=submit name=choice class="button" value=Markdistribution>
      <input type=submit name=choice class="button" value=Scorecard>
  <br><br><br>
  <p style="color:white">Answer sheet evaluation</p>
  <input type=submit name=choice class="button" value=Evaluate>
    </form>
    </body>
    '''

Add exam


In [33]:
@app.route('/addexam', methods=['GET', 'POST'])
#register for teacher only
def addexam():
    if request.method == 'POST':
      try:
        addexam.id=request.form['examid']
        #id=int(addexam.id)
        sem=request.form['sem']
        subid=request.form['subid']
        m=request.form['mark']
        #mark=int(m)
        addexam.noofqs=int(request.form['noofqs'])
        print(addexam.id,subid,m,sem)
        addexam.n=[]
        for i in range(addexam.noofqs):
          addexam.n.append(i+1)
        with sqlite3.connect("employee.db") as con:  
                cur = con.cursor()  
                cur.execute("INSERT into exam_db values (?,?,?,?,?)",(addexam.id,subid,m,sem,addexam.noofqs))  
                row_set=cur.execute("SELECT * from exam_db") 
                rows = cur.fetchall()
                for row in rows:
                  print(row)  
                #cur1 = con.cursor()
                #cur1.execute("INSERT into question_db values (?,?,?,?,?)",(addexam.id,addqs.noofqs,qs,modelans,maxmark))
                #row_set=cur1.execute("SELECT * from question_db where examid = ?",(addexam.id,)) 
                #rows = cur1.fetchall()
                #for row in rows:
                  #print(row) 
                msg = "Exam successfully Added" 
                con.commit() 

      except:  
        con.rollback()  
        msg = "We can not add the Exam to the list"  
      finally:
        print(msg)
        if request.form['submit']=="AddExam":
          return redirect(url_for('addqs'))#change to viewexam
        con.close()  
        
      #add to database 
    return '''
    <!doctype html>
    
    <title>Add Exam</title>
    <center>
    <br><br><br><br><br><br><br><br><br><br>
    <h2 style="color:white">Add Exam </h1>
     <style>
      .button {
  background-color: #4CAF50;
  border: none;
  color: white;
  padding: 15px 32px;
  text-align: center;
  text-decoration: none;
  display: inline-block;
  font-size: 16px;
  margin: 4px 2px;
  cursor: pointer;
}

input[type=text] {
  width: 10%;
  padding: 12px 20px;
  margin: 8px 0;
  box-sizing: border-box;
}
input[type=integer] {
  width: 10%;
  padding: 12px 20px;
  margin: 8px 0;
  box-sizing: border-box;
}
body {
  background-image: url('https://www.creativefabrica.com/wp-content/uploads/2020/03/08/Back-to-school-background-as-blank-copy-Graphics-3411986-1.jpg');
  background-repeat: no-repeat;
  background-attachment: fixed;  
  background-size: cover;
  font-family: 'Courier New', monospace;
}
</style>
<body>
    <form method=post enctype=multipart/form-data>
    <br><br>
      <input type=integer name=examid placeholder=ExamID>
      <input type=text name=sem placeholder=Semester>
      <input type=text name=subid placeholder=SubjectID>
      <input type=integer name=mark placeholder=Totalmark>
      <input type=integer name=noofqs placeholder=No.of questions><br><br>
      <input type=submit name=submit class="button" value=AddExam> 
    </form>
    <br><br>  
    </body>
    '''  

Add Question

In [34]:
@app.route('/addqs', methods=['GET', 'POST'])
#register for teacher only
def addqs():
    if request.method == 'POST':
      try:
        id=int(addexam.id)  #examid from previous page
        q=request.form['qno']
        qno=int(q)          #qno
        qs=request.form['qs']#question
        modelans=request.form['modelans'] #modelans
        m=request.form['maxmark'] 
        maxmark=int(m)  #maxmark
        with sqlite3.connect("employee.db") as con:  
                cur = con.cursor()  
                cur.execute("INSERT into question_db values (?,?,?,?,?)",(id,qno,qs,modelans,maxmark))  
                msg = "Question successfully Added" 
                con.commit() 

      except:  
          con.rollback()  
          msg = "We can not add the Questions to the list"  
      finally:
          print(msg)
          if request.form['submit']=="AddQuestion":
            return redirect(url_for('addqs'))
          con.close()  
         
    return '''
    <!doctype html>
    <title>Add Exam</title>
    <center><br><br><br><br><br><br><br><br><br><br>

    <h2 style="color:white">Add Exam </h1>
     <style>
      .button {
  background-color: #4CAF50;
  border: none;
  color: white;
  padding: 15px 32px;
  text-align: center;
  text-decoration: none;
  display: inline-block;
  font-size: 16px;
  margin: 4px 2px;
  cursor: pointer;
}
input[type=integer] {
  width: 30%;
  padding: 12px 20px;
  margin: 8px 0;
  box-sizing: border-box;
}
input[type=text] {
  width: 30%;
  padding: 12px 20px;
  margin: 8px 0;
  box-sizing: border-box;
}
body {
  background-image: url('https://www.creativefabrica.com/wp-content/uploads/2020/03/08/Back-to-school-background-as-blank-copy-Graphics-3411986-1.jpg');
  background-repeat: no-repeat;
  background-attachment: fixed;  
  background-size: cover;
  font-family: 'Courier New', monospace;
}
</style>
<body>
    <form method=post enctype=multipart/form-data>
      <input type=integer name=qno placeholder=Q.NO>
      <input type=text name=qs placeholder=Question>

      <input type=integer name=modelans placeholder=Model answer>
      <input type=integer name=maxmark placeholder=Maxmark>
      <input type=submit name=submit class="button" value=AddQuestion> 
    </form>
    <br><br>  
    </body>
    '''  

To add a student

In [35]:
@app.route('/addstud', methods=['GET', 'POST'])
#register for teacher only
def addstud():
#rollno,name,sem,pwd (totmark)
    if request.method == 'POST':
      try:
        addstud.studname=request.form['studname']
        rno=request.form['rno']
        sem=request.form['sem']
        studpwd=request.form['studpwd']
        print(addstud.studname,rno,sem,studpwd)
        with sqlite3.connect("employee.db") as con:  
                cur = con.cursor()  
                cur.execute("INSERT into Student (rno, studname, sem, studpwd) values (?,?,?,?)",(rno,addstud.studname,sem,studpwd))  
                con.commit()  
                msg = "Student successfully Added"  
      except:  
        con.rollback()  
        msg = "We can not add the Student to the list"  
      finally:
        if request.form['submit']=="AddStudent":
          return redirect(url_for('view'))
        con.close()
      #add to database
      
    return '''
    <!doctype html>
    <title>Add Student</title>
    <center>
<br><br><br><br><br><br><br><br><br><br>
    <h1 style="color:white">Add Student </h1>
         <style>
      .button {
  background-color: #4CAF50;
  border: none;
  color: white;
  padding: 15px 32px;
  text-align: center;
  text-decoration: none;
  display: inline-block;
  font-size: 16px;
  margin: 4px 2px;
  cursor: pointer;
}

input[type=text] {
  width: 20%;
  padding: 12px 20px;
  margin: 8px 0;
  box-sizing: border-box;
}
body {
  background-image: url('https://www.creativefabrica.com/wp-content/uploads/2020/03/08/Back-to-school-background-as-blank-copy-Graphics-3411986-1.jpg');
  background-repeat: no-repeat;
  background-attachment: fixed;  
  background-size: cover;
  font-family: 'Courier New', monospace;
}
</style>
<body>
    <form method=post enctype=multipart/form-data>
    <br><br>
      <input type=text name=studname placeholder=StudentName>
      <input type=text name=rno placeholder=RollNumber>
      <input type=text name=sem placeholder=Semester>
      <input type=text name=studpwd placeholder=Password><br><br>
      <input type=submit name=submit class="button" value=AddStudent>
    </form>
    </body>
    '''

Extracted text from hadwritten answer sheet stored into database

In [36]:


def allowed_file(filename):
    return '.' in filename and \
           filename.rsplit('.', 1)[1].lower() in ALLOWED_EXTENSIONS

@app.route('/upload', methods=['GET', 'POST'])

def upload():
    if request.method == 'POST':
      try:
        
        # check if the post request has the file part
        if 'file' not in request.files:
            flash('No file part')
            return redirect(request.url)
        file = request.files['file']
        # if user does not select file, browser also
        # submit an empty part without filename
        if file.filename == '':
            flash('No selected file')
            return redirect(request.url)
        if file and allowed_file(file.filename):
            upload.filename = secure_filename(file.filename)
            #print("Uploaded",register.studname,"filename :",upload_file.filename)
            file.save(os.path.join(app.config['UPLOAD_FOLDER'], upload.filename))
            answer=handrecog(upload.filename)
            print("Answer:",answer)
            upload.answer=""
            k=1
            for i in answer:
              upload.answer=upload.answer+i+"*" 
              cur.execute("INSERT into score_db (rollno, qno, examid, answer, mark, evaluated) values (?,?,?,?,0,'false')",(login.rno, k, student.examid, i));
              k+=1
            #with sqlite3.connect("employee.db") as con:  
              #cur = con.cursor()  
            cur.execute("INSERT into answer_db (rollno, examid, sem, studmark) values (?,?,?,0)",(login.rno, student.examid, login.sem));
            msg = "Answer successfully Added" 
            print(msg)
            con.commit() 
      except:  
        con.rollback()  
        msg = "We can not add the Answer to the list"  
        print(msg)
      finally:
        if request.form['submit']=="Next":
          return redirect(url_for('u'))
        con.close()
    return render_template("questionview.html", rows=student.rows, examid=student.examid)
    #return '''
    #<!doctype html>
    #<title>Upload Answer Sheet</title>
    #<h1>Upload Answer Sheet</h1>
    #<form method=post enctype=multipart/form-data>
     #   <input type=submit name=submit value=Next>
    #</form>
    #'''
    
    

**Score calculation**

Pre installations

In [37]:
!pip install -U spacy
!python -m spacy download en_core_web_lg
!python -m spacy download en_core_web_sm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 6.2 MB 8.7 MB/s 
     |████████████████████████████████| 181 kB 50.8 MB/s 
     |████████████████████████████████| 660 kB 35.8 MB/s 
     |████████████████████████████████| 42 kB 1.2 MB/s 
     |████████████████████████████████| 457 kB 57.7 MB/s 
     |████████████████████████████████| 10.1 MB 38.7 MB/s 
     |████████████████████████████████| 58 kB 5.3 MB/s 
  Attempting uninstall: typing-extensions
    Found existing installation: typing-extensions 4.2.0
    Uninstalling typing-extensions-4.2.0:
      Successfully uninstalled typing-extensions-4.2.0
  Attempting uninstall: catalogue
    Found existing installation: catalogue 1.0.0
    Uninstalling catalogue-1.0.0:
      Successfully uninstalled catalogue-1.0.0
  Attempting uninstall: srsly
    Found existing installation: srsly 1.0.5
    Uninstalling srsly-1.0.5:
      Successfully uninstalled srs

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 400.7 MB 6.2 kB/s 
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 12.8 MB 7.5 MB/s 
  Attempting uninstall: en-core-web-sm
    Found existing installation: en-core-web-sm 2.2.5
    Uninstalling en-core-web-sm-2.2.5:
      Successfully uninstalled en-core-web-sm-2.2.5
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [38]:
pip install rake-nltk

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.5 MB 7.8 MB/s 
     |████████████████████████████████| 749 kB 46.4 MB/s 
  Attempting uninstall: regex
    Found existing installation: regex 2019.12.20
    Uninstalling regex-2019.12.20:
      Successfully uninstalled regex-2019.12.20
  Attempting uninstall: nltk
    Found existing installation: nltk 3.2.5
    Uninstalling nltk-3.2.5:
      Successfully uninstalled nltk-3.2.5


In [39]:
pip install pytextrank

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.1 MB 11.8 MB/s 
     |████████████████████████████████| 46 kB 3.8 MB/s 
     |████████████████████████████████| 38.1 MB 1.2 MB/s 
     |████████████████████████████████| 11.2 MB 12.4 MB/s 
     |████████████████████████████████| 930 kB 38.6 MB/s 
  Attempting uninstall: scipy
    Found existing installation: scipy 1.4.1
    Uninstalling scipy-1.4.1:
      Successfully uninstalled scipy-1.4.1
  Attempting uninstall: pygments
    Found existing installation: Pygments 2.6.1
    Uninstalling Pygments-2.6.1:
      Successfully uninstalled Pygments-2.6.1
  Attempting uninstall: matplotlib
    Found existing installation: matplotlib 3.2.2
    Uninstalling matplotlib-3.2.2:
      Successfully uninstalled matplotlib-3.2.2
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.8.4
    Uninstalling graphviz-0.8.4:
      Successfully uni

In [2]:
%cd /content/drive/MyDrive/handwritten-text-recognition

/content/drive/MyDrive/handwritten-text-recognition


In [47]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [42]:
#To count number of questions
def qs_count(path):
    count = 0
    with open(path) as lines:
        for count, l in enumerate(lines, start=1):
            pass
    return count 

Importing the functions for scorecalculation

In [3]:
import score1 as s

Evaluation

In [8]:
@app.route('/score', methods=['GET', 'POST'])
#register for teacher only
def score():
  import sqlite3
  conn = sqlite3.connect('employee.db')  
  c = conn.cursor() 
  exam_set=conn.execute('''SELECT * from exam_db ''');
  for row in exam_set:
    examid=row[0]
    sem=row[3]
    rollno_set=conn.execute('''SELECT * from Student where sem=? ''',(sem, ));
    
    for row in rollno_set:
      roll=row[0]
      print("Roll number of student",roll)
      sum=0
      answer_set=conn.execute('''SELECT * from score_db where rollno = ? and examid = ? and evaluated='false' ''',(roll,examid,));
      for ans in answer_set:
        test_ans = ans[3]   #student answer
        q=str(ans[2])       #question number
        model_set=conn.execute('''SELECT modelans,maxmark from question_db where examid=? and qno=? ''',(examid,q));
        print("Answer for question ",q," - ",test_ans)
        for model in model_set:
          key_ans = model[0]
        maxmark = model[1]


        grouped_keys = s.group(s.extract_keywords(key_ans))
        grouped_keys = s.remove_duplicates(grouped_keys)
        final_keywords = s.finalize(grouped_keys)
        answer_key = s.dictionarize(final_keywords, key_ans)
        kw = s.extract_keywords(test_ans)
        final_kw = s.finalize(s.remove_duplicates(s.group(kw)))
        answer_test = s.dictionarize(final_kw, test_ans)
        group_final = s.remove_duplicates(s.group(final_keywords))
        new_final = []
        for i in group_final:
          if len(i) > 1:
            for j in i:
              for k in i:
                  if j != k and j in k:
                      new_final.append(j)
          else:
            new_final.append(i[-1])
        marks = 0
        for i in answer_key:
          marks += len(answer_key[i])
          #print(marks) 
        test_score, kw_match = s.score(answer_key, answer_test)
        #print(test_score)
        test_score = test_score/marks*maxmark
# print(kw_match)
        if (test_score % 1) > 0.5:
          rem = 1
        else:
          rem = 0
  #print(l)
        m = int(test_score)+rem
        print("Mark for this is - ",m)    #mark for 1 qstn
        sum+=m

    #DB storing mark
        q=int(q)
        cursor = conn.cursor()
        sql = ''' UPDATE score_db
              SET mark = ?, where qno = ? and rollno = ? and examid = ?'''
              #evaluated='true' not set 
        data = (m, q, roll, examid)
        cur = conn.cursor()
        cur.execute(sql,data)
        conn.commit()
  #end of one student
      print("Total mark of the student : ",sum)
      print("_______________________________")
  #store into DB
      cursor = conn.cursor()
      sql = ''' UPDATE answer_db
              SET studmark = ? where rollno = ? and examid = ?'''
      data = (sum, roll, examid)
      cur = conn.cursor()
      cur.execute(sql,data)
      conn.commit()
    print("!!! Evaluation is successfully completed !!!") 
    print()

  print("TOTAL MARK OF STUDENTS")
  rollno_set=conn.execute('''SELECT examid,rollno,studmark from answer_db where examid=?''', (examid,));
  for row in rollno_set:
    print(row)
  print("   END OF AN EVALUATION   ")
  conn.commit
  #clear this exam from db
  #sql_update_query = """DELETE from exam_db where examid = ?"""
  #c.execute(sql_update_query, (examid,))
  #print("Record deleted successfully")
  print()
  print("END OF ALL EVALUATIONS")
  return redirect(url_for('teacher'))
  conn.commit()

In [9]:
@app.route('/evaluate', methods=['GET', 'POST'])
def answerdis_file():
    #return send_from_directory(app.config['UPLOAD_FOLDER'],filename)
    return render_template('upload.html')

In [10]:
@app.route("/view")  
def view():  
    con = sqlite3.connect("employee.db")  
    con.row_factory = sqlite3.Row  
    cur = con.cursor()  
    cur.execute("select * from Student")  
    rows = cur.fetchall()  
    return render_template("view.html",rows = rows)
con.commit()   

NameError: ignored

In [11]:
app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://c1ff-34-125-190-27.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [01/Jun/2022 19:18:28] "GET / HTTP/1.1" 404 -
127.0.0.1 - - [01/Jun/2022 19:18:29] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [01/Jun/2022 19:19:31] "GET / HTTP/1.1" 404 -
127.0.0.1 - - [01/Jun/2022 19:19:31] "GET /favicon.ico HTTP/1.1" 404 -


In [49]:
from flask import Flask,render_template
from pyngrok import ngrok
os.environ["FLASK_ENV"]="development"
app = Flask(__name__)
port=8080

ngrok.set_auth_token("29zPmdNRFoMfdXFQhrBcN12eNC1_2ZynwGCs9WPX4VpWM4iUC")
  # Start ngrok when app is run
public_url=ngrok.connect(port).public_url
app.config["BASE_URL"]=public_url
@app.route("/")
def hello():
    return "Hello World!"
threading.Thread(target=app.run,kwargs={"use_reloader": False}).start()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: development
 * Debug mode: on


Exception in thread Thread-46:
Traceback (most recent call last):
  File "/usr/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.7/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/local/lib/python3.7/dist-packages/flask/app.py", line 990, in run
    run_simple(host, port, self, **options)
  File "/usr/local/lib/python3.7/dist-packages/werkzeug/serving.py", line 1010, in run_simple
    inner()
  File "/usr/local/lib/python3.7/dist-packages/werkzeug/serving.py", line 963, in inner
    fd=fd,
  File "/usr/local/lib/python3.7/dist-packages/werkzeug/serving.py", line 806, in make_server
    host, port, app, request_handler, passthrough_errors, ssl_context, fd=fd
  File "/usr/local/lib/python3.7/dist-packages/werkzeug/serving.py", line 699, in __init__
    HTTPServer.__init__(self, server_address, handler)
  File "/usr/lib/python3.7/socketserver.py", line 452, in __init__
    self.server_bind()
  File 

In [48]:
ngrok.kill()


In [ ]:
#!unzip '/content/drive/MyDrive/gbw-ebb1a287.zip' -c 

In [ ]:
#delete images folder
#!find . -maxdepth 1 -type f -name "*.jpeg" -exec rm '/content/drive/MyDrive/images/';
!rm -r "/content/drive/MyDrive/images/"

In [ ]:
import sqlite3

conn = sqlite3.connect('/content/drive/MyDrive/my_db/evaluation_db.db')  # You can create a new database by changing the name within the quotes
c = conn.cursor() # The database will be saved in the location where your 'py' file is saved


r_set=conn.execute('''SELECT * from exam_db''');
print("exam_db")
for row in r_set:
    print(row)

Text answer stored into DATABASE table named answer_db